# Forecasting - Exercícios
## Semana 2
### Prof. Dr. Neylson Crepalde

Aqui estão algumas anotações e exercícios de forecasting baseados no curso *Sequences, Time Series and Prediction* do Coursera.

### **SUMÁRIO**
>1. [1 - Preparando features e labels](#features)

<font size="6"><a id="features">1 - Preparando features e labels</a></font>
***